# setting up

In [1]:
# info about current GPU
!nvidia-smi

Mon Oct 28 10:07:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  |   00000000:05:00.0  On |                  N/A |
|  0%   49C    P0             62W /  270W |    1005MiB /   8192MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Setting up the environment & base path of project
import os
HOME = os.getcwd()
print(HOME)

# Installing ultralytics
%pip install ultralytics

from IPython.display import display, Image, clear_output
clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO


Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 827.4/930.9 GB disk)


In [ ]:
# installing roboflow, this is where our dataset is located
%pip install roboflow

# importing the .env file for the api key
from dotenv import load_dotenv
load_dotenv()
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

# downloading the dataset
from roboflow import Roboflow
rf = Roboflow(ROBOFLOW_API_KEY)
project = rf.workspace("nathanneve").project("nematode-v1")
version = project.version(1)
dataset = version.download("yolov11")


#### installation of pytorch, make sure the version of pytorch corresponds to the locally installed version of cuda
in this case cuda version needs to be 12.4

In [5]:
# pytorch is nessesary for gpu acceleration with jupyter notebooks
# https://pytorch.org/get-started/locally/
# nvcc --version -> checks installed version of CUDA


# https://stackoverflow.com/questions/75103127/getting-notimplementederror-could-not-run-torchvisionnms-with-arguments-fr
# %pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu124

import torch
print("CUDA Available: ", torch.cuda.is_available())  # Expect True
if torch.cuda.is_available():
    print("Current CUDA Device Index: ", torch.cuda.current_device())
    print("CUDA Device Name: ", torch.cuda.get_device_name(0))
else:
    print("CUDA device not accessible.")

import torchvision
print("torchvision CUDA: ", torchvision.ops.nms is not None)



CUDA Available:  True
Current CUDA Device Index:  0
CUDA Device Name:  NVIDIA GeForce RTX 3070
torchvision CUDA:  True


### Training

In [6]:
# command if training on google collab
# yolo task=detect mode=train model=yolov11n.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 plots=True
from ultralytics import YOLO

my_data = f"{dataset.location}/data.yaml"
model = YOLO("yolo11n.pt")
results = model.train(data=my_data, epochs=25, imgsz=640, model="yolov11n.pt")
# reference https://docs.ultralytics.com/modes/train/

Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov11n.pt, data=c:\_repos_\2425-FW-001-Aaltjes-tellen\code\Nematode-v1-1/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

train: Scanning C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Nematode-v1-1\train\labels.cache... 5769 images, 1162 backgrounds, 0 corrupt: 100%|██████████| 5769/5769 [00:00<?, ?it/s]
val: Scanning C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Nematode-v1-1\valid\labels.cache... 396 images, 86 backgrounds, 0 corrupt: 100%|██████████| 396/396 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25      2.49G      1.478      2.043      1.152         22        640: 100%|██████████| 361/361 [01:04<00:00,  5.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  4.11it/s]

                   all        396        782      0.822      0.735       0.81      0.498


...

      25/25      2.38G      1.086     0.6386      1.018         17        640: 100%|██████████| 361/361 [00:55<00:00,  6.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  5.40it/s]

                   all        396        782      0.789      0.878      0.864      0.667



25 epochs completed in 0.415 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 5.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  4.29it/s]


                   all        396        782      0.789      0.877      0.864      0.667
                   egg        267        587      0.927      0.934      0.947      0.741
                 shell        129        195      0.651      0.821      0.781      0.593
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train


### Validation

In [9]:
%cd {HOME}
# command if training on google collab
# yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml
from ultralytics import YOLO

'''
⚠ NOTE: selections do not prevent you from specifying a combination for a model that doesn't exist.
Reference the documentation for valid model specifications: https://docs.ultralytics.com/models
'''
my_data = f"{dataset.location}/data.yaml"
model_path = f"{HOME}/runs/detect/train/weights/best.pt"
model = YOLO(model_path)
results = model.val(data=my_data)
# reference https://docs.ultralytics.com/modes/val/

c:\_repos_\2425-FW-001-Aaltjes-tellen\code
Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Nematode-v1-1\valid\labels.cache... 396 images, 86 backgrounds, 0 corrupt: 100%|██████████| 396/396 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:03<00:00,  6.98it/s]


                   all        396        782      0.788      0.877      0.864      0.666
                   egg        267        587      0.926      0.934      0.946       0.74
                 shell        129        195      0.651      0.821      0.781      0.592
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\val


### Testing

In [ ]:
%cd {HOME}
# command if training on google collab
# yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

src = f"{dataset.location}/test/images"
print("Model names:", getattr(model, 'names', "No names attribute found"))
from ultralytics import YOLO

model = YOLO(model_path)
results = model.predict(source=src, conf=0.25)
# save=True parameter saves the results to runs/detect/predict
# reference https://docs.ultralytics.com/models/rtdetr/

In [24]:
import glob
from IPython.display import Image, display

src = "images"
model = YOLO(model_path)
print("Model path exists:", os.path.exists(model_path))

from ultralytics import YOLO

results = model.predict(source=src, conf=0.25, save=True)

Model path exists: True

image 1/4 c:\_repos_\2425-FW-001-Aaltjes-tellen\code\images\DSC_0171.jpg: 640x448 (no detections), 14.5ms
image 2/4 c:\_repos_\2425-FW-001-Aaltjes-tellen\code\images\DSC_0181.jpg: 640x448 1 shell, 12.2ms
image 3/4 c:\_repos_\2425-FW-001-Aaltjes-tellen\code\images\DSC_0185.jpg: 640x448 (no detections), 12.3ms
image 4/4 c:\_repos_\2425-FW-001-Aaltjes-tellen\code\images\DSC_0230.jpg: 448x640 (no detections), 17.9ms
Speed: 3.1ms preprocess, 14.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict10
